## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
%matplotlib inline

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C://Users//Zach Pausa//PythonStuff//WeatherPy//Weather_database//WeatherPy.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,-7.58,78,75,9.22,broken clouds
1,1,Atuona,PF,-9.8000,-139.0333,78.67,73,3,16.75,clear sky
2,2,Fallon,US,39.4735,-118.7774,60.28,35,20,4.61,few clouds
3,3,Cape Town,ZA,-33.9258,18.4232,67.08,62,0,6.91,clear sky
4,4,Mataura,NZ,-46.1927,168.8643,59.49,83,100,3.85,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Atuona,PF,-9.8000,-139.0333,78.67,73,3,16.75,clear sky
8,8,East London,ZA,-33.0153,27.9116,70.38,81,14,4.59,few clouds
9,9,Carnarvon,AU,-24.8667,113.6333,75.27,78,80,4.61,broken clouds
10,10,Bambous Virieux,MU,-20.3428,57.7575,77.25,94,40,2.30,scattered clouds
13,13,Panguna,PG,-6.3164,155.4848,75.36,82,85,1.43,light rain
14,14,Vaini,TO,-21.2000,-175.2000,81.50,80,68,21.56,broken clouds
17,17,San Policarpo,PH,12.1791,125.5072,78.89,83,100,15.68,light rain
21,21,Galle,LK,6.0367,80.2170,75.63,87,29,7.78,scattered clouds
23,23,Mahajanga,MG,-15.7167,46.3167,78.82,83,40,1.14,scattered clouds
25,25,Barranca,PE,-10.7500,-77.7667,73.65,67,74,8.43,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                259
City                   259
Country                256
Lat                    259
Lng                    259
Max Temp               259
Humidity               259
Cloudiness             259
Wind Speed             259
Current Description    259
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                256
City                   256
Country                256
Lat                    256
Lng                    256
Max Temp               256
Humidity               256
Cloudiness             256
Wind Speed             256
Current Description    256
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,78.67,clear sky,-9.8000,-139.0333,
8,East London,ZA,70.38,few clouds,-33.0153,27.9116,
9,Carnarvon,AU,75.27,broken clouds,-24.8667,113.6333,
10,Bambous Virieux,MU,77.25,scattered clouds,-20.3428,57.7575,
13,Panguna,PG,75.36,light rain,-6.3164,155.4848,
14,Vaini,TO,81.50,broken clouds,-21.2000,-175.2000,
17,San Policarpo,PH,78.89,light rain,12.1791,125.5072,
21,Galle,LK,75.63,scattered clouds,6.0367,80.2170,
23,Mahajanga,MG,78.82,scattered clouds,-15.7167,46.3167,
25,Barranca,PE,73.65,broken clouds,-10.7500,-77.7667,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)  
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,78.67,clear sky,-9.8000,-139.0333,Villa Enata
8,East London,ZA,70.38,few clouds,-33.0153,27.9116,Tu Casa
9,Carnarvon,AU,75.27,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
10,Bambous Virieux,MU,77.25,scattered clouds,-20.3428,57.7575,Casa Tia Villa
13,Panguna,PG,75.36,light rain,-6.3164,155.4848,
14,Vaini,TO,81.50,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
17,San Policarpo,PH,78.89,light rain,12.1791,125.5072,Agapita M. Nicart
21,Galle,LK,75.63,scattered clouds,6.0367,80.2170,Amangalla
23,Mahajanga,MG,78.82,scattered clouds,-15.7167,46.3167,Restaurant Greedy Coco Lodge Majunga
25,Barranca,PE,73.65,broken clouds,-10.7500,-77.7667,El Olivar Hospedaje


In [10]:
import numpy as np
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,78.67,clear sky,-9.8000,-139.0333,Villa Enata
8,East London,ZA,70.38,few clouds,-33.0153,27.9116,Tu Casa
9,Carnarvon,AU,75.27,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
10,Bambous Virieux,MU,77.25,scattered clouds,-20.3428,57.7575,Casa Tia Villa
13,Panguna,PG,75.36,light rain,-6.3164,155.4848,NaN
14,Vaini,TO,81.50,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
17,San Policarpo,PH,78.89,light rain,12.1791,125.5072,Agapita M. Nicart
21,Galle,LK,75.63,scattered clouds,6.0367,80.2170,Amangalla
23,Mahajanga,MG,78.82,scattered clouds,-15.7167,46.3167,Restaurant Greedy Coco Lodge Majunga
25,Barranca,PE,73.65,broken clouds,-10.7500,-77.7667,El Olivar Hospedaje


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,78.67,clear sky,-9.8000,-139.0333,Villa Enata
8,East London,ZA,70.38,few clouds,-33.0153,27.9116,Tu Casa
9,Carnarvon,AU,75.27,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
10,Bambous Virieux,MU,77.25,scattered clouds,-20.3428,57.7575,Casa Tia Villa
14,Vaini,TO,81.50,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
17,San Policarpo,PH,78.89,light rain,12.1791,125.5072,Agapita M. Nicart
21,Galle,LK,75.63,scattered clouds,6.0367,80.2170,Amangalla
23,Mahajanga,MG,78.82,scattered clouds,-15.7167,46.3167,Restaurant Greedy Coco Lodge Majunga
25,Barranca,PE,73.65,broken clouds,-10.7500,-77.7667,El Olivar Hospedaje
26,Port Alfred,ZA,70.03,overcast clouds,-33.5906,26.8910,The Halyards Hotel


In [12]:
# 8a. Create the output File (CSV)
output_data_file =  ("C://Users//Zach Pausa//PythonStuff//WeatherPy//Weather_database//WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))